In [90]:
from get_data import *
from get_dataframe import *
from get_urls import *
import pandas as pd
import numpy as np
from preprocess import *

In [91]:
submission_df = pd.read_csv('../data/all_submissions.csv')
submission_df = pd.DataFrame(submission_df, columns=['id', 'title', 'selftext', 'subreddit', 'created_utc'])
comment_df = pd.read_csv('../data/comment.csv')
comment_df = pd.DataFrame(comment_df, columns=['id','title','selftext','subreddit','created_utc','body'])
comment_df = comment_df.drop(columns=['selftext', 'title'])
comment_df

,id,subreddit,created_utc,body
0,hdysfhh,insomnia,1632406331,&gt; It takes me anywhere between 5-8 hours to...
1,hdyqj2t,insomnia,1632405494,"Yea, it's tough you should try and manage it."
2,hdyq6b3,insomnia,1632405336,One what? One pill? One pill can contain 10 mg...
3,hdyq3p8,insomnia,1632405304,Try and get a prescription for Prazosin from y...
4,hdypirx,insomnia,1632405042,"yeah i have tried everything, it just sucks be..."
...,...,...,...,...
10094,h7gr2h7,insomnia,1627932966,[removed]
10095,h7gqg2n,insomnia,1627932692,"I know this may be different for everyone, but..."
10096,h7gqcbx,insomnia,1627932648,This is probably one of the best replies I hav...
10097,h7gq4w1,insomnia,1627932559,"I understand, I was in the same boat as you go..."


In [92]:
submission_df = submission_df.dropna(subset=['selftext'])
submission_df = submission_df.loc[submission_df['selftext'] != '[removed]'].loc[submission_df['selftext'] != '[deleted]']
comment_df = comment_df.dropna(subset=['body'])
comment_df = comment_df.loc[comment_df['body'] != '[removed]'].loc[comment_df['body']!='[deleted]']
comment_df

,id,subreddit,created_utc,body
0,hdysfhh,insomnia,1632406331,&gt; It takes me anywhere between 5-8 hours to...
1,hdyqj2t,insomnia,1632405494,"Yea, it's tough you should try and manage it."
2,hdyq6b3,insomnia,1632405336,One what? One pill? One pill can contain 10 mg...
3,hdyq3p8,insomnia,1632405304,Try and get a prescription for Prazosin from y...
4,hdypirx,insomnia,1632405042,"yeah i have tried everything, it just sucks be..."
...,...,...,...,...
10093,h7grqmf,insomnia,1627933260,can you please mention what other lifestyle ch...
10095,h7gqg2n,insomnia,1627932692,"I know this may be different for everyone, but..."
10096,h7gqcbx,insomnia,1627932648,This is probably one of the best replies I hav...
10097,h7gq4w1,insomnia,1627932559,"I understand, I was in the same boat as you go..."


In [93]:
p = Preprocess()

In [94]:
submission_df.dtypes

id             object
title          object
selftext       object
subreddit      object
created_utc     int64
dtype: object

In [95]:
submission_df['selftext'] = submission_df['selftext'].apply(lambda x: p.preprocess_status(x))
submission_df['title'] = submission_df['title'].apply(lambda x: p.preprocess_status(x))
comment_df['body'] = comment_df['body'].apply(lambda x: p.preprocess_status(x))
comment_df


,id,subreddit,created_utc,body
0,hdysfhh,insomnia,1632406331,> it take me anywher between 5-8 hour to fall ...
1,hdyqj2t,insomnia,1632405494,"yea , it' tough you should tri and manag it ."
2,hdyq6b3,insomnia,1632405336,one what ? one pill ? one pill can contain 10 ...
3,hdyq3p8,insomnia,1632405304,tri and get a prescript for prazosin from your...
4,hdypirx,insomnia,1632405042,"yeah i have tri everyth , it just suck becaus ..."
...,...,...,...,...
10093,h7grqmf,insomnia,1627933260,can you pleas mention what other lifestyl chan...
10095,h7gqg2n,insomnia,1627932692,"i know thi may be differ for everyon , but wha..."
10096,h7gqcbx,insomnia,1627932648,thi is probabl one of the best repli i have go...
10097,h7gq4w1,insomnia,1627932559,"i understand , i wa in the same boat as you go..."


In [96]:
def is_substring(status, drug_list):
    result=False
    for drug in drug_list:
        if drug in status:
            result = True
    return result

def generate_drugs_statistics(drug_type:str, drug_list, df, col_name):
    drug_list = [p.preprocess_status(x) for x in drug_list]
    df[drug_type] = df[col_name].apply(lambda x: is_substring(x, drug_list))

In [100]:
drug_dict = {'Doxepin': ['Doxepin', 'Silenor'],
            'Estazolam': ['Estazolam'],
            'Eszopiclone': ['Eszopiclone ', 'Lunesta'],
            'Ramelteon': ['Rozerem', 'Ramelteon'],
            'Temazepam': ['Temazepam', 'Restoril'],
            'Triazolam': ['Triazolam', 'Halcion'],
            'Zaleplon': ['Zaleplon', 'Sonata'],
            'Zolpidem': ['Zolpidem', 'Ambien', 'Edluar', 'Intermezzo', 'Zolpimist'],
            'Zaleplon': ['Zaleplon', 'Sonata'],
            'Suvorexant': ['Suvorexant ', 'Belsomra']}

treatment_dict = {'Herb': ['Herb', 'valerian', 'Chamomile', 'Ashwagandha', 'Kava'],
                 'Melatonin' : ['Melatonin'],
                 'Acupunture': ['acupunture'],
                 'Relaxation': ['relaxation', 'exercise', 'yoga', 'pilates', 'meditation', 'biofeedback'],
                 'CBD': ['CBD', 'cannabis'],
                 'Sleep hypnosis': ['hypnosis', 'hypnotherapy']}

for drug in drug_dict:
    generate_drugs_statistics(drug, drug_dict[drug], comment_df, 'body')
    generate_drugs_statistics(drug, drug_dict[drug], submission_df, 'selftext')
    
for treatment in treatment_dict:
    generate_drugs_statistics(treatment, treatment_dict[treatment], comment_df, 'body')
    generate_drugs_statistics(treatment, treatment_dict[treatment], submission_df, 'selftext')
    
# print(submission_df.columns)
# print(comment_df.columns)

Index(['id', 'title', 'selftext', 'subreddit', 'created_utc', 'Doxepin',
       'Estazolam', 'Eszopiclone', 'Ramelteon', 'Temazepam', 'Triazolam',
       'Zaleplon', 'Zolpidem', 'Suvorexant', 'Herb', 'Melatonin', 'Acupunture',
       'Relaxation', 'CBD', 'Sleep hypnosis'],
      dtype='object')
Index(['id', 'subreddit', 'created_utc', 'body', 'Doxepin', 'Estazolam',
       'Eszopiclone', 'Ramelteon', 'Temazepam', 'Triazolam', 'Zaleplon',
       'Zolpidem', 'Suvorexant', 'Herb', 'Melatonin', 'Acupunture',
       'Relaxation', 'CBD', 'Sleep hypnosis'],
      dtype='object')


In [102]:
columns = ['Doxepin','Estazolam', 'Eszopiclone', 'Ramelteon', 'Temazepam', 'Triazolam',
       'Zaleplon', 'Zolpidem', 'Suvorexant', 'Herb', 'Melatonin', 'Acupunture',
       'Relaxation', 'CBD', 'Sleep hypnosis']

In [104]:
for col in columns:
    print(col)
    print('comment_df:', len(comment_df.loc[comment_df[col] == True]))
    print('submission_df:', len(submission_df.loc[submission_df[col] == True]))
    print('\n')

Doxepin
comment_df: 19
submission_df: 111


Estazolam
comment_df: 1
submission_df: 5


Eszopiclone
comment_df: 78
submission_df: 343


Ramelteon
comment_df: 6
submission_df: 37


Temazepam
comment_df: 35
submission_df: 197


Triazolam
comment_df: 9
submission_df: 21


Zaleplon
comment_df: 20
submission_df: 73


Zolpidem
comment_df: 381
submission_df: 1357


Suvorexant
comment_df: 17
submission_df: 101


Herb
comment_df: 85
submission_df: 621


Melatonin
comment_df: 437
submission_df: 3259


Acupunture
comment_df: 1
submission_df: 0


Relaxation
comment_df: 537
submission_df: 3039


CBD
comment_df: 111
submission_df: 542


Sleep hypnosis
comment_df: 20
submission_df: 95




In [109]:
list(submission_df['selftext'])

["i suffer from anxieti and depress sinc last month . i wa prescrib lexapro low dose but didn't like the side effect and stop take the lexapro . i would take activan as need if my anxieti got realli bad . the sleep issu start about 4 week ago . i would onli get 5-6 hour sleep and wake up in the middl of the night . i wa fine dure the first part of last week fall asleep ad wake u after 5-6 hr . last friday i got up at 3:30 am unabl to fall back asleep , follow night slept 3 hr . the last two night i would doze off watch tv like i normal do . turn the tv off and went to bed . toss and turn wide awak , unabl to sleep . i feel like a zombi becaus i'm exhaust . if i tri to nap dure the day , i sort of doze off but my bodi won ' let me sleep . & #x200b ; i have a prescript of low dose klonopin which i hear can be use as a sleep aid . i don't want to get addict to the pill . i'm do to see the doctor later today and ask about other sleep aid that might be effect . i think i suffer from sleepi 

In [119]:
from sutime import SUTime
test = '5 day awak and it keep happen everi ten to 15 day i feel like i am gone for real thi time ani advic ? i dont know what to do anymor it so frustrat and i cant keep up like thi , everi night is a pain'
sutime = SUTime(mark_time_ranges=True, include_range=True)
sutime.parse(test)

[{'timex-value': 'P5D',
  'start': 0,
  'end': 5,
  'text': '5 day',
  'type': 'DURATION',
  'value': 'P5D'},
 {'start': 36,
  'end': 49,
  'text': 'ten to 15 day',
  'type': 'DURATION',
  'value': 'P10D/P15D'},
 {'timex-value': '2021-09-25TNI',
  'start': 183,
  'end': 188,
  'text': 'night',
  'type': 'TIME',
  'value': '2021-09-25TNI'}]